In [26]:
#%matplotlib widget
#%matplotlib inline
#from IPython.display import display, HTML, Image
#display(HTML("<style>.con\tainer { width:90% !important; }</style>"))
#from pynx.cdi.widgets import CDIViewer
from ipywidgets import interact, IntSlider
#import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [27]:
# carregando o dado
with h5py.File('../dataset/summed.cxi', 'r') as file:
    d = file['entry_1/data_1/data'][()] #Acessa o conjunto de dados dentro do arquivo HDF5. Os dados são armazenados em d.

def iplot(i):
    plt.clf()
    plt.imshow(d[i], norm=LogNorm(), cmap='jet')
    plt.colorbar()
    plt.title(f'2D Slice at Index {i}')
    plt.tight_layout()
    plt.show()

# Set up the interactive slider
slider = IntSlider(min=0, max=len(d)-1, step=1, value=len(d)//2, continuous_update=False)

# Create the interactive plot
interact(iplot, i=slider)
print(d.shape)

interactive(children=(IntSlider(value=75, continuous_update=False, description='i', max=150), Output()), _dom_…

(151, 256, 256)


In [28]:
with h5py.File('../dataset/summed.cxi', 'r') as file:
    d = np.array(file['entry_1/data_1/data']) #Acessa o conjunto de dados dentro do arquivo HDF5. Os dados são armazenados em d.

print('Image Dimension:')
print(f'Height............:  {d.shape[0]}')
print(f'Width.............:  {d.shape[1]}')
print(f'Depth.............:  {d.shape[2]}')

Image Dimension:
Height............:  151
Width.............:  256
Depth.............:  256


In [29]:
# Pre processing
# Steps:
#  1 -> flatfield
#  2 -> mask

def pre_processing(d):
    def aplicar_flatfield_e_plotar(d):
        # Criar o flatfield (cria um array que tem a mesma forma e tipo de dados que a matriz
        #tridimensional d, onde todos os elementos são defindos como 1. 
        flatfield = np.ones_like(d) 

        # Aplicar o flatfield a todas as fatias
        dados_corrigidos = d / flatfield

        #Calcula o número de fatias no conjunto de dados tridimensionais onde o eixo 0 represeta
        #as fatias. Pega o tamanho ao longo do eixo 0.
        num_fatias = d.shape[0]
        
        return dados_corrigidos

    def aplicar_mascara_e_plotar(dados_corrigidos):
        num_fatias = dados_corrigidos.shape[0]

        for i in range(num_fatias):
            # Gerar uma máscara binária aleatória para cada fatia
            mask = np.random.randint(0, 2, size=dados_corrigidos[i, :, :].shape)

            # Aplicar a máscara: onde a máscara é 1, substitui por -1
            dados_corrigidos[i, :, :][mask == 1] = -1

            # Aplicar a máscara à fatia selecionada
            dados_mascarados = dados_corrigidos[i, :, :] * mask

    # Chamar a função de processamento
    dados_corrigidos = aplicar_flatfield_e_plotar(d)

    # Chamar a função de aplicar máscara na variável modificada
    aplicar_mascara_e_plotar(dados_corrigidos)

pre_processing(d)


In [ ]:
### next step - pynx (ER + 